## Xpath解析原理
1）实例化一个etree对象，且需要将被解析的页面源码数据加载到该对象中。

2）调用etree对象中的Xpath方法结合Xpath表达式实现标签的定位和内容的获取。

## 实例化一个etree对象（两种情况：本地与网站）
首先，先导入lxml中的etree包。

（1）第一种情况：将本地的html文档中的源码数据加载到etree对象中； tree = etree.parse(filepath)

（2）第二种情况：将从互联网上获取到的源码数据加载到该对象中；tree = etree.HTML(text)

## Xpath表达式
（1）/：单斜杠从根节点开始定位，表示一个层级；比如：'/html/body/div'，一层一层递进；

（2）//：双斜杠可以从任意位置开始定位，表示多个层级；比如：'//div'，直接跳到指定位置；

（3）属性定位：//div[@class='xxx']  其中，div为网页的块状标签，class为该div标签的唯一属性值；

（4）索引定位：//div[@class='xxx']/p[3] 直接跳到class为'xxx'的div标签，再跳下一层第三个p标签（索引从1开始）；

（5）取文本：/text() 获取的是标签中直系的文本内容；//text() 获取的是标签中非直系的文本内容（指定标签下所有的文本内容）；

（6）取属性：/@attrName  （attrName为属性名，比如：图片img/src）

In [ ]:
#/text()例：
r = tree.xpath('//div[@class="tang"]//li[5]/a/text()')[0]
#获取 class为'tang'的div标签 --> 第5个li标签 --> a标签 --> 文本内容列表的第一个元素

#//text()例：
r = tree.xpath('//li[7]//text()') #获取第7个li标签下的所有文本内容

#取属性
r = tree.xpath('//div[@class="song"]/img/src')

## time.sleep()：设置访问时间间隔
很多网站的反爬虫机制都设置了访问间隔时间，一个IP如果短时间内超过了指定的次数就会进入“冷却CD”，所以除了轮换IP和user_agent，可以设置访问时间间隔，让间隔长一些，比如每抓取一个页面休眠一个随机时间：

相对来说，这是一个比较可靠的做法。 因为本来爬虫就可能会给对方网站造成访问的负载压力，所以这种防范既可以从一定程度上防止被封，还可以降低网站的访问压力。如果访问过于频繁，有些网站会直接封掉IP，让你再也无法访问其数据。所以为了保险起见，最好设置下睡眠时间。

那么如何设置访问时间间隔呢？代码很简单，只需在爬取时加到循环里就可以。

In [ ]:
import time
import random
time.sleep(random.randint(5,10)) 
#具体的随机数，可以自行设置，太小的话，作用不大，太大的话，代码运行时间会加长;

In [ ]:
# 第一页的url
url = "https://www.sephora.cn/category/230168-230166-60001/page1/?hasInventory=0&sortField=1&sortMode=desc"
 
# 多页时，只需加入循环
for i in range(12): #最多爬取12页，每页25个商品
    url = "https://www.sephora.cn/category/230168-230166-60001/page{}/?hasInventory=0&sortField=1&sortMode=desc".format(i*25)

In [2]:
## 导入相关包
import pandas as pd
import requests
import random
import time
import csv
import re 
from lxml import etree # xpath解析
 
## 创建文件对象
f = open('丝芙兰商品数据.csv', 'w', encoding='utf-8-sig', newline="") 
csv_write = csv.DictWriter(f, fieldnames=['品牌', '商品名', '价格'])
csv_write.writeheader() # 写入文件头

## 从代理IP池，随机获取一个IP，比如必须ProxyPool项目在运行中
def get_proxy():
    try:
        PROXY_POOL_URL = 'http://localhost:5555/random'
        response = requests.get(PROXY_POOL_URL)
        if response.status_code == 200:
            return response.text
    except ConnectionError:
        return None
    
## 循环爬取
for i in range(12): #最多爬取12页
    url = "https://www.sephora.cn/category/230168-230166-60001/page{}/?hasInventory=0&sortField=1&sortMode=desc".format(i)
    # request请求获取网页页面
    page_text = requests.get(url=url).text
    # etree解析HTML文档
    tree = etree.HTML(page_text)
 
    # 获取品牌
    brand = tree.xpath("//div[@class='p_cont']//div[@class='p_brandEn']/text()")
    # 获取商品
    product = tree.xpath("//div[@class='p_cont']//div[@class='p_productCN']/a/text()")
    # 获取价格
    price = tree.xpath("//div[@class='p_cont']//div[@class='p_discount commonFontPrice']/text()")
 
    # 调整价格范围问题
    t1 = pd.DataFrame(price).reset_index()
    t21 = t1[~t1.loc[:, 0].str.contains('~')]
    t22 = t1[t1.loc[:, 0].str.contains('~')].copy()
    t22.loc[:, 'index'] -= 1
    t3 = pd.merge(t21, t22, how='left', left_on='index', right_on='index')
    t3.fillna('', inplace=True)
    price = list(t3['0_x'] + t3['0_y'])

    # 去除评论日期的换行符及空格
    # comment_date = list(map(lambda date: re.sub('\s+', '', date), comment_date))  # 去掉换行符制表符
    # comment_date = list(filter(None, comment_date))  # 去掉上一步产生的空元素
 
    for j in range(12):
        data_dict = {'品牌': brand[j], '商品名': product[j], '价格': price[j]}
        csv_write.writerow(data_dict)
 
    print('第{}页爬取成功'.format(i+1))
 
    # 设置睡眠时间间隔，防止频繁访问网站
    time.sleep(random.randint(10, 15))

print("---------------")
print("所有评论爬取成功")

第1页爬取成功
第2页爬取成功
第3页爬取成功
第4页爬取成功
第5页爬取成功
第6页爬取成功
第7页爬取成功
第8页爬取成功
第9页爬取成功
第10页爬取成功
第11页爬取成功
第12页爬取成功
---------------
所有评论爬取成功
